# Point Extraction of WRF Climate Data
## Introduction
### Purpose
The purpose of this Jupyter Notebook is to extract data from WRF data sets at two latitude and longitude coordinate pairs provided by Kannon Lee at PND Engineers. Kannon's application is thermal analysis, so we'll extract the following variables at **hourly** resolution:
 - `t2`: Two meter air temperature (Kelvin)

The following **hourly** datasets will be queried for each variable:
 - The ERA-Interim Reanalysis Historical Baseline (1979 - 2015)
 - The NCAR-CCSM4 Historical Model Run (1970 - 2005)
 - The NCAR-CCSM4 RCP 8.5 Model Run (2006 - 2100)
 - The GFDL-CM3 Historical Model Run (1970 - 2005)
 - The GFDL-CM3 RCP 8.5 Model Run (2006 - 2100)

 
### Objective
Create a minimalist dataset of temperature data suitable for ingest to Kannon's thermal analysis. The data product will constitute several CSV files organized by location, temperature variable, and the particular reanalysis / climate model / emissions scenario combination. Files with data in units Kelvin and degrees Centigrade will be provided for convenience.

### Background
The data set for extraction is: [Historical and Projected Dynamically Downscaled Climate Data for the State of Alaska and surrounding regions at 20km spatial resolution and hourly temporal resolution](http://ckan.snap.uaf.edu/dataset/historical-and-projected-dynamically-downscaled-climate-data-for-the-state-of-alaska-and-surrou)

Spatial resolution of these data is 20 km, and the temporal resolution is hourly, with daily summaries also available. For the more detail regarding the source data, read the description and linked journal articles available via the link above. 

## Methods & Implementation
The extraction will be implemented via this Jupyter Notebook which provides an interactive way to execute Python code in a web browser. The extraction code, as well as any necessary narrative and documention, will be woven together and self-contained here. The cells are intended to be executed top to bottom in sequential order.

### Technical Details
This notebook is being developed on an *Atlas* compute node. *Atlas* is the machine that serves as SNAP's primary high performance computing cluster. The Python (3.8) environment is managed by `pipenv` inside a `conda` shell. However, this code is portable and should run on virtually any machine where a modern Python executable (> 3.6) is available.

### Extraction Steps for each Variable / Data set.
1. Identify filepaths from which to extract the data.
2. Transform lat-lon WGS84 coordinates to match the WRF grid.
3. Extract the data from the grid cell corresponding to the desired coordinates.
4. Store the results in a DataFrame object where each row is a time record, and each column is the value of the variable for that time at a particular location.
5. Concatentate these data and store in dictionary object (i.e., hash table) so they can be retreived 

### Quality Control Steps for Each Extraction
1. The volume of data in play here is quite large. It is probable that some records are incomplete. We'll examine each extraction for missing values and handle them by ensuring that:
    - Missing values have consistent representation.
    - A record exists where missing values are left intact (the time series is complete)
    - A record exists where missing values are dropped (the time series has a gap, but all records are valid)
2. Perform a bias adjustment via the delta (difference-of-means) method.
3. Round to appropriate precision levels.
4. Convert from Kelvin to degrees Celsius.

### Data Export
1. Prepare self-describing file names for each variable / location / model-or-scenario combination.
2. Write minimal (no index) tabular .csv files for each of the above.

In [1]:
# import geospatial data libraries
import xarray as xr
import numpy as np
import pandas as pd
import multiprocessing as mp
import warnings
from pathlib import Path
from pyproj import Transformer
from pyproj.crs import CRS
from functools import partial
warnings.filterwarnings("ignore", category=FutureWarning) 

In [2]:
"""This code block will set up a few global variables we'll use throughout the extraction."""

# parameters for extracting in parallel
PROCESSES = 32

# coordinates provided via a KMZ file from Kannon Lee
wgs84_coords = {
                "Chefornak": (60.158787283199999, -164.281157796799988),
                "Ambler": (67.08675982, -157.8569733),
               }

# climate variables of interest
climate_vars = ["t2"]

# paths to local copy of the data
hourly_basepath = Path("/rcs/project_data/wrf_data/hourly_s3_t2_fetch_02042022/")

# empty list to track netCDF files that have missing information
files_with_missing_info = []

# initialize empty nested dictionary object for results
climate_data_di = {}
for var in climate_vars:
    climate_data_di[var] = {}

In [3]:
"""This code block defines functions to fetch the paths to the netCDF files
containing the WRF outputs for the variables and combinations of interest."""


def fetch_target_directory(basepath, climate_variable):
    """Get directory containing data for desired temporal resolution and variable.
    Args:
        basepath (Path object): daily or hourly basepath
        climate_variable (str): climate variable abberviation (e.g. `t2min`)
    Returns:
        target_dir (PosixPath object): directory of data for extraction
    """
    target_dir = basepath.joinpath(climate_variable)
    return target_dir


def fetch_target_filepaths(target_dir, filename_match):
    """Get list of netCDF files matching some pattern.
    Args:
        target_dir (PosixPath object): directory of data for extraction
        filename_match (str): tag found in the netCDF filenames (e.g. `rcp85`)
    Returns:
        target_fps (list): list of netCDF files from which to extract data
    """
    target_fps = [fp for fp in target_dir.glob(f"*{filename_match}*.nc")]
    return target_fps


"""This code block defines functions to extract and store the actual data from netCDF files
for a particular set of geospatial coordinates."""


def get_data(fp, var):
    """Extract data for multiple coordinates from a single netCDF file.
    Args:
        fp (PosixPath object): path to netCDF file.
        var (str): climate variable abberviation (e.g. `t2min`)
    Returns:
        df (DataFrame object): extraction results
    """
    with xr.open_dataset(fp) as ds:
        
        # project WGS84 coordinates using proj string from WRF file
        wrf_proj_str = ds.attrs["proj_parameters"]
        wrf_crs = CRS.from_proj4(wrf_proj_str)
        transformer = Transformer.from_crs("epsg:4326", wrf_crs)
        wrf_coords = {
            p_name: transformer.transform(*coords)
            for p_name, coords in wgs84_coords.items()
        }

        # query xarray dataset for nearest cell to each coordinate
        try:
            temp_data = {
                p_name: ds[var].sel(xc=coords[0], yc=coords[1], method="nearest").values
                for p_name, coords in wrf_coords.items()
            }
            # make DataFrame where time series is the rows, location temp data are columns
            df = pd.DataFrame(temp_data, index=ds.time.values)
        except:
            # if there is an error, log the filename and use np.nan to represent no data
            files_with_missing_info.append(fp)
            temp_data = {p_name: np.nan for p_name in wrf_coords.keys()}
            df = pd.DataFrame(temp_data, index = ds.time.values)
        return df


def get_data_from_all_filepaths(filepaths, var):
    """Extract data for multiple coordinates from many netCDF files. This function calls the `get_data`
    function defined earlier.
    Args:
        filepaths (list): list of netCDF files.
        var (str): climate variable abbreviation (e.g. `t2min`)
    Returns:
        all_temp_data (DataFrame object): aggregated extraction results
    """
    list_of_temp_data_from_single_files = []
    
    for fp in filepaths:
        temp_data_from_one_file = get_data(fp, var)
        list_of_temp_data_from_single_files.append(temp_data_from_one_file)
    
    all_temp_data = pd.concat(list_of_temp_data_from_single_files)
    return all_temp_data



def mp_extract(filepaths, var):
    # Set up and run multiprocessing step.
    partial_function = partial(
        get_data_from_all_filepaths,
        var=var,
    )
    with mp.Pool(PROCESSES) as pool:
        extracted = pool.map(partial_function, filepaths)
        pool.close()
        pool.join()
        pool = None
    return extracted


#### Notes on the above extraction code

The WRF coordinate reference system is obtained from the proj4 string in the netCDF dataset attributes. Then the `pyproj.Transformer` is used to project the coordinates to the CRS used in the `xarray` dataset. 

The `xarray.DataSet` object has a `.sel` method to query the data by location. The parameter `method="nearest"` must be set to query the nearest grid cell to the specified input coordinates, otherwise the method will look for the the specified coordinate exactly as provided and likely fail. 

In [4]:
"""This code block defines functions to run the extraction."""

def run_hourly_extraction():
    """Extract hourly data for `t2` and store results in the pre-initialized global-scope dict.
    Args: None
    Returns: None
    """
    # set the filepaths
    target_dir = hourly_basepath
    hourly_ccsm4_fps = fetch_target_filepaths(target_dir, "CCSM4")
    hourly_cm3_fps = fetch_target_filepaths(target_dir, "CM3")
    hourly_era_fps = fetch_target_filepaths(target_dir, "ERA")
    
    cm3_historical_model_fps = [x for x in hourly_cm3_fps if "historical" in x.name]
    cm3_rcp85_fps = [x for x in hourly_cm3_fps if "rcp85" in x.name]
    
    ccsm4_historical_model_fps = [x for x in hourly_ccsm4_fps if "historical" in x.name]
    ccsm4_rcp85_fps = [x for x in hourly_ccsm4_fps if "rcp85" in x.name]
    
    
    # do the extraction
    climate_data_di["t2"]["hourly_reanalysis"] = get_data_from_all_filepaths(hourly_era_fps, "t2")
    climate_data_di["t2"]["hourly_cm3_rcp85"] = get_data_from_all_filepaths(cm3_rcp85_fps, "t2")
    climate_data_di["t2"]["hourly_historical_modeled_cm3"] = get_data_from_all_filepaths(cm3_historical_model_fps, "t2")
    climate_data_di["t2"]["hourly_ccsm4_rcp85"] = get_data_from_all_filepaths(ccsm4_rcp85_fps, "t2")
    climate_data_di["t2"]["hourly_historical_modeled_ccsm4"] = get_data_from_all_filepaths(ccsm4_historical_model_fps, "t2")


#### Running the extraction

We'll now run the extraction and do a brief inspection of the results.

In [5]:
run_hourly_extraction()

In [6]:
climate_data_di

{'t2': {'hourly_reanalysis':                       Chefornak      Ambler
  2000-01-01 00:00:00  246.339005  242.985001
  2000-01-01 01:00:00  246.010193  242.712189
  2000-01-01 02:00:00  245.326569  242.498566
  2000-01-01 03:00:00  245.003372  242.477371
  2000-01-01 04:00:00  244.790939  242.388931
  ...                         ...         ...
  2011-12-31 19:00:00  245.346069  238.243057
  2011-12-31 20:00:00  245.102936  238.821930
  2011-12-31 21:00:00  244.439743  239.694748
  2011-12-31 22:00:00  244.690506  240.280502
  2011-12-31 23:00:00  244.843750  240.666748
  
  [322800 rows x 2 columns],
  'hourly_cm3_rcp85':                       Chefornak      Ambler
  2009-01-01 00:00:00  263.653564  251.504562
  2009-01-01 01:00:00  263.407196  251.639191
  2009-01-01 02:00:00  263.269318  251.636307
  2009-01-01 03:00:00  263.406952  252.072937
  2009-01-01 04:00:00  263.432556  254.918564
  ...                         ...         ...
  2099-12-31 19:00:00  281.116302  271.423309
 

In [7]:
# check which netCDF files may have been missing information
print(len(files_with_missing_info))
print(files_with_missing_info)

0
[]


In [8]:
"""This code block will execute the bias correction process."""

def compute_mean(df, location, time_start, time_stop):
    return df[location].loc[time_start:time_stop].mean()


def compute_bias(modeled_mean, observed_mean):
    return modeled_mean - observed_mean


def apply_bias(df, location, bias):
    return df[location] + bias


def bias_adjust_data():
    for place in wgs84_coords.keys():

        era_mean = compute_mean(climate_data_di["t2"]["hourly_reanalysis"],
                                place, "1980-01-01", "2009-12-31")
        cm3_historical_mean = compute_mean(climate_data_di["t2"]["hourly_historical_modeled_cm3"],
                                place, "1980-01-01", "2009-12-31")
        ccsm4_historical_mean = compute_mean(climate_data_di["t2"]["hourly_historical_modeled_ccsm4"],
                                place, "1980-01-01", "2009-12-31")
        
        cm3_bias = compute_bias(cm3_historical_mean, era_mean)
        ccsm4_bias = compute_bias(ccsm4_historical_mean, era_mean)

        bias_adjusted_cm3 = apply_bias(climate_data_di["t2"]["hourly_cm3_rcp85"], place, cm3_bias)
        bias_adjusted_ccsm4 = apply_bias(climate_data_di["t2"]["hourly_ccsm4_rcp85"], place, ccsm4_bias)
                
        climate_data_di["t2"]["hourly_cm3_rcp85"][place + "_bias_adjusted"] = bias_adjusted_cm3
        climate_data_di["t2"]["hourly_ccsm4_rcp85"][place + "_bias_adjusted"] = bias_adjusted_ccsm4

bias_adjust_data()       

In [9]:
# confirm the bias was applied
climate_data_di["t2"]["hourly_cm3_rcp85"]

,Chefornak,Ambler,Chefornak_bias_adjusted,Ambler_bias_adjusted
2009-01-01 00:00:00,263.653564,251.504562,263.902069,251.950760
2009-01-01 01:00:00,263.407196,251.639191,263.655701,252.085388
2009-01-01 02:00:00,263.269318,251.636307,263.517822,252.082504
2009-01-01 03:00:00,263.406952,252.072937,263.655457,252.519135
2009-01-01 04:00:00,263.432556,254.918564,263.681061,255.364761
...,...,...,...,...
2099-12-31 19:00:00,281.116302,271.423309,281.364807,271.869507
2099-12-31 20:00:00,281.045807,271.832825,281.294312,272.279022
2099-12-31 21:00:00,281.080811,272.181824,281.329315,272.628021
2099-12-31 22:00:00,281.173798,272.472809,281.422302,272.919006


In [19]:
"""This code block will tidy up the outputs by converting from K to C, rounding to two decimals,
and doing some column renaming. It will also write the results to disk in the form of .csv files."""

def convert_Kelvin_to_degC(k):
    
    c = round(k - 273.15, 2)
    return c

for k in climate_data_di["t2"].keys():
    #climate_data_di["t2"][k] = climate_data_di["t2"][k].applymap(convert_Kelvin_to_degC)
    #climate_data_di["t2"][k]["time"] = climate_data_di["t2"][k].index.values
    if "rcp85" in k:
        out_df = climate_data_di["t2"][k][["time", "Chefornak_bias_adjusted", "Ambler_bias_adjusted"]]
        out_df.rename(columns = {"Chefornak_bias_adjusted": "Chefornak 2 m temperature (degrees C)",
             "Ambler_bias_adjusted": "Ambler 2 m temperature (degrees C)"}, inplace=True)
        out_df.to_csv("wrf_" + k + "_t2.csv", index=False)
    else:
        out_df = climate_data_di["t2"][k][["time", "Chefornak", "Ambler"]]
        out_df.rename(columns = {"Chefornak": "Chefornak 2 m temperature (degrees C)",
             "Ambler": "Ambler 2 m temperature (degrees C)"}, inplace=True)
        out_df.to_csv("wrf_" + k + "_t2.csv", index=False)


/tmp/ipykernel_4979/2195503503.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  out_df.rename(columns = {"Chefornak_bias_adjusted": "Chefornak 2 m temperature (degrees C)",
/tmp/ipykernel_4979/2195503503.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  out_df.rename(columns = {"Chefornak_bias_adjusted": "Chefornak 2 m temperature (degrees C)",


In [22]:
df = pd.read_csv("wrf_hourly_ccsm4_rcp85_t2.csv")
df

,time,Chefornak 2 m temperature (degrees C),Ambler 2 m temperature (degrees C)
0,2034-01-01 00:00:00,1.06,-9.48
1,2034-01-01 01:00:00,0.88,-9.04
2,2034-01-01 02:00:00,0.59,-8.30
3,2034-01-01 03:00:00,0.37,-6.73
4,2034-01-01 04:00:00,0.12,-5.34
...,...,...,...
832147,2099-12-31 19:00:00,3.29,-2.65
832148,2099-12-31 20:00:00,3.29,-2.63
832149,2099-12-31 21:00:00,3.39,-1.27
832150,2099-12-31 22:00:00,3.41,-1.14


In [23]:
df = pd.read_csv("wrf_hourly_cm3_rcp85_t2.csv")
df

,time,Chefornak 2 m temperature (degrees C),Ambler 2 m temperature (degrees C)
0,2009-01-01 00:00:00,-9.25,-21.20
1,2009-01-01 01:00:00,-9.49,-21.06
2,2009-01-01 02:00:00,-9.63,-21.07
3,2009-01-01 03:00:00,-9.49,-20.63
4,2009-01-01 04:00:00,-9.47,-17.79
...,...,...,...
832166,2099-12-31 19:00:00,8.21,-1.28
832167,2099-12-31 20:00:00,8.14,-0.87
832168,2099-12-31 21:00:00,8.18,-0.52
832169,2099-12-31 22:00:00,8.27,-0.23


In [25]:
df = pd.read_csv("wrf_hourly_historical_modeled_ccsm4_t2.csv")
df

,time,Chefornak 2 m temperature (degrees C),Ambler 2 m temperature (degrees C)
0,1971-01-01 00:00:00,-7.08,-13.92
1,1971-01-01 01:00:00,-7.57,-11.90
2,1971-01-01 02:00:00,-8.36,-11.16
3,1971-01-01 03:00:00,-8.94,-10.95
4,1971-01-01 04:00:00,-9.37,-10.60
...,...,...,...
315307,2001-12-31 19:00:00,0.93,-32.49
315308,2001-12-31 20:00:00,0.86,-28.76
315309,2001-12-31 21:00:00,0.78,-25.92
315310,2001-12-31 22:00:00,0.66,-24.43


In [26]:
df = pd.read_csv("wrf_hourly_cm3_rcp85_t2.csv")
df

,time,Chefornak 2 m temperature (degrees C),Ambler 2 m temperature (degrees C)
0,2009-01-01 00:00:00,-9.25,-21.20
1,2009-01-01 01:00:00,-9.49,-21.06
2,2009-01-01 02:00:00,-9.63,-21.07
3,2009-01-01 03:00:00,-9.49,-20.63
4,2009-01-01 04:00:00,-9.47,-17.79
...,...,...,...
832166,2099-12-31 19:00:00,8.21,-1.28
832167,2099-12-31 20:00:00,8.14,-0.87
832168,2099-12-31 21:00:00,8.18,-0.52
832169,2099-12-31 22:00:00,8.27,-0.23


In [27]:
df = pd.read_csv("wrf_hourly_reanalysis_t2.csv")
df

,time,Chefornak 2 m temperature (degrees C),Ambler 2 m temperature (degrees C)
0,2000-01-01 00:00:00,-26.81,-30.16
1,2000-01-01 01:00:00,-27.14,-30.44
2,2000-01-01 02:00:00,-27.82,-30.65
3,2000-01-01 03:00:00,-28.15,-30.67
4,2000-01-01 04:00:00,-28.36,-30.76
...,...,...,...
322795,2011-12-31 19:00:00,-27.80,-34.91
322796,2011-12-31 20:00:00,-28.05,-34.33
322797,2011-12-31 21:00:00,-28.71,-33.46
322798,2011-12-31 22:00:00,-28.46,-32.87


http://ckan.snap.uaf.edu/dataset/projected-monthly-and-derived-temperature-products-771m-cmip3-ar4/resource/99a8e100-84e6-433d-8293-176e12dd8c1e